In [ ]:
import os
import sqlite3
import subprocess
from github import Github

conn = sqlite3.connect('test_results.db')

conn.execute('''
CREATE TABLE IF NOT EXISTS results
(id INTEGER PRIMARY KEY AUTOINCREMENT, 
module TEXT NOT NULL, 
exercise TEXT NOT NULL, 
result TEXT NOT NULL);
''')

conn.commit()
conn.close
g = Github()  
def compile_and_test(module, repo_name, repo_owner):

    repo = g.get_user(repo_owner).get_repo(repo_name)
    #add logic to check not duplicated
    subprocess.check_call(f'git clone https://github.com/{repo_owner}/{repo_name}.git', shell=True)

    # Navigate into the repository
    os.chdir(repo_name)

    # For each exercise in the module
    for content in repo.get_contents(module):
        if content.type == "dir":
            exercise = content.name
            os.chdir(exercise)
            
            # Compile the C code
                        
            
            # Save the result in the database
            conn.execute('''
            INSERT INTO results (module, exercise, result) 
            VALUES (?, ?, ?);
            ''', (module, exercise, 'Pass' if result == 0 else 'Fail'))
            
            # Navigate back up to the module directory
            os.chdir('..')
            
    # Commit the changes to the database
    conn.commit()

    # Navigate back to the original directory
    os.chdir('..')

    # Remove the cloned repository
    subprocess.check_call(f'rm -rf {repo_name}', shell=True)

compile_and_test('module1', 'repo_name')  # replace 'module1' and 'repo_name' with your module and repo names


In [1]:
import sqlite3

conn = sqlite3.connect('test_results.db')

conn.execute('''
CREATE TABLE IF NOT EXISTS grades
(id INTEGER PRIMARY KEY AUTOINCREMENT, 
module INT NOT NULL, 
exercise INT NOT NULL, 
result INT NOT NULL);
''')